# Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException 
import time

import pandas as pd
import numpy as np

# extracting locations to search

In [2]:
data_cities=pd.read_csv(r"USCities - City_list.csv")

In [3]:
data_cities.head()

,City,State short,State full,County
0,Anchorage,AK,Alaska,ANCHORAGE
1,Jber,AK,Alaska,ANCHORAGE
2,Indian,AK,Alaska,ANCHORAGE
3,Kongiganak,AK,Alaska,BETHEL
4,Adak,AK,Alaska,ALEUTIANS WEST


In [16]:
unique_city=data_cities['City'].unique()
unique_county=data_cities['County'].unique()

In [17]:
#type(unique_city)
final_locations_all=np.concatenate([unique_city,unique_county])

In [20]:
len(final_locations)

20576

In [19]:
final_locations=np.unique(final_locations_all)

In [ ]:
#final_locations contains all locations

In [7]:
final_req=pd.read_csv('final_req.csv')
final_req.head()#contains 2 columns

,Fact,Xpath
0,"Population estimates, July 1, 2019, (V2019)","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody..."
1,"Population estimates base, April 1, 2010, (V2...","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody..."
2,"Population, percent change - April 1, 2010 (es...","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody..."
3,"Population, Census, April 1, 2010","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody..."
4,"Persons under 5 years, percent","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody..."


In [9]:
empty_list=[]
for i in range(50):
    empty_list.append('NaN')

# Scrap data

In [10]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(chrome_options=options, executable_path=r'chromedriver_new.exe')
driver.get('https://www.census.gov/quickfacts/fact/table/US/PST045219')


D:\PIAIC\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#driver = webdriver.Chrome(r"chromedriver_new.exe")
#driver.get('https://www.census.gov/quickfacts/fact/table/US/PST045219')

In [12]:
driver.refresh()

In [13]:
no_suggestion_list=[]
problem_list=[]
no_data_found=[]
default_us_url='https://www.census.gov/quickfacts/fact/table/US/PST045219'
no_suggestion_msg=r"Can't find what you're looking for? Re-enter the first 2 characters of your geography or zip and select from the dropdown list"
no_suggestion_list.append(no_suggestion_msg)

# Funtions

In [14]:
def check_locations(location):
    
    global driver
    
    clear_button()
    try:
        inputElement = WebDriverWait(driver, 5).until(lambda x: x.find_element_by_id('qf-search-box'))
    except TimeoutException:
        clear_button()
        inputElement = WebDriverWait(driver, 10).until(lambda x: x.find_element_by_id('qf-search-box'))
    inputElement.send_keys(location,Keys.DOWN)
    try:
        options = WebDriverWait(driver, 50).until(lambda x: x.find_elements_by_class_name("ui-menu-item-wrapper"))#this pick all auto suggestions
    except TimeoutException:        
        clear_button()
        try:
            inputElement.send_keys(location,Keys.DOWN)
        except:
            clear_button()
            inputElement.send_keys(location,Keys.DOWN)
        options = WebDriverWait(driver, 60).until(lambda x: x.find_elements_by_class_name("ui-menu-item-wrapper"))#this pick all auto suggestions
    return options

def input_button(location):
    global driver
    xpath_of_search_box='//*[@id="qf-search-box"]'
    clear_button()
    inputElement = WebDriverWait(driver, 60).until(lambda x: x.find_element_by_id('qf-search-box'))
    inputElement.send_keys(location)
    #WebDriverWait(driver, 30).until(lambda driver: driver.find_element_by_xpath(xpath_of_search_box).text.strip() == '')
    time.sleep(1.8)
    #driver.implicitly_wait(3)
    inputElement.send_keys(Keys.ENTER)
    #time.sleep(0.5)

def clear_button():
    global driver
    try:
        cross_button=WebDriverWait(driver, 60).until(lambda x: x.find_element_by_id("qf-nav-link-clear"))
        cross_button.click()
    except :
        #time.sleep()
        driver.refresh()
        cross_button=WebDriverWait(driver, 60).until(lambda x: x.find_element_by_id("qf-nav-link-clear"))
        cross_button.click()
    #time.sleep(0.5)

def get_data(location):    
    global empty_list,final_req,driver
    #clear_button()
    input_button(location)
    final_req[location]= empty_list
    #time.sleep(1.5)
    WebDriverWait(driver, 20).until(EC.text_to_be_present_in_element((By.ID, "qfcontent"), location))#waits until page loads
    for entity in final_req['Fact']:
        
        final_req.at[final_req['Fact']==entity,location]=(WebDriverWait(driver, 10).until(lambda x: x.find_element_by_xpath(final_req.loc[final_req['Fact']==entity]['Xpath'].values[0]))).text
    final_req[location].replace(regex=True, inplace=True, to_replace=r'[\ue840\n\ue83f\n]', value=r'')
    
def put_null(location):
    global empty_list,final_req
    location=location.strip()
    final_req[location]= empty_list#put nan


# Main

In [117]:
for var in range(0,100,10):#from 0 to index 100
    start_index=var
    end_index=var+10
    for location in final_locations[start_index:end_index]:
        print(location)
        print('==========')
        options=check_locations(location)
        list_of_options=[]

        for i in options:
            list_of_options.append(i.text)

        if list_of_options==no_suggestion_list:#means no city
            #print('nikal')
            put_null(location)
            no_data_found.append(location)
        else:
            for real_location in list_of_options:
                if real_location not in final_req.columns:#means it in not already in data
                    try:
                        get_data(real_location)
                    except: 
                        problem_list.append(real_location)
                        
                        continue
    

88
AARP
ABAC
ABBEVILLE
ABMPS
ACADIA
ACCOMACK
ADA
ADAIR
ADAMS
ADDISON
AIKEN
AITKIN
ALACHUA
ALAMANCE
ALAMEDA
ALAMOSA
ALBANY
ALBEMARLE
ALCONA
ALCORN
ALEUTIANS EAST
ALEUTIANS WEST
ALEXANDER
ALEXANDRIA CITY
ALFALFA
ALGER
ALLAMAKEE
ALLEGAN
ALLEGANY
ALLEGHANY
ALLEGHENY
ALLEN
ALLENDALE
ALPENA
ALPINE
AMADOR
AMELIA
AMF
AMHERST
AMITE
ANCHORAGE
ANDERSON
ANDREW
ANDREWS
ANDROSCOGGIN
ANGELINA
ANNE ARUNDEL
ANOKA
ANSON
ANTELOPE
ANTRIM
APACHE
APG
APPANOOSE
APPLING
APPOMATTOX
ARANSAS
ARAPAHOE
ARCHER
ARCHULETA
ARENAC
ARKANSAS
ARLINGTON
ARMSTRONG
AROOSTOOK
ARP
ARTHUR
ASCENSION
ASHE
ASHLAND
ASHLEY
ASHTABULA
ASOTIN
ASSUMPTION
ASU
ATASCOSA
ATCHISON
ATHENS
ATKINSON
ATLANTIC
ATOKA
ATTALA
AUDRAIN
AUDUBON
AUGLAIZE
AUGUSTA
AURORA
AUSTIN
AUTAUGA
AVERY
AVOYELLES
Aaronsburg
Abbeville
Abbot
Abbotsford
Abbott
Abbottstown
Abbyville
Abell


In [92]:
#for var in range(2450,5000,10):#from 0 to index 100#now start from 858
#start_index=5
#end_index=7
for location in columns_with_nan[:]:
    print(location ,columns_with_nan.index(location))
    
    print('==========')
    options=check_locations(location)
    list_of_options=[]

    for i in options:
        list_of_options.append(i.text)

    if list_of_options==no_suggestion_list:#means no city
        
        no_data_found.append(location)
    else:
        for real_location in list_of_options:
            if real_location not in final_req.columns:#means it in not already in data
                if not 'Puerto Rico'in real_location:#means no piero data
                    try:    
                        get_data(real_location)
                    except: 
                        problem_list.append(real_location)
                        continue    


Anthony city, New Mexico 0
Arthur County, Nebraska 1
Ballard County, Kentucky 2
Banner County, Nebraska 3
Latrobe borough, Pennsylvania 4
Lawrence County, Kentucky 5
Lee County, Kentucky 6
Bedford town, Virginia 7
Bella Vista city, Arkansas 8
Blaine County, Nebraska 9
Borden County, Texas 10
Bracken County, Kentucky 11
Briscoe County, Texas 12
Brookhaven city, Georgia 13
Calhoun County, West Virginia 14
Carlisle County, Kentucky 15


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=86.0.4240.111)


In [22]:
#for var in range(5000,7500,10):#from 0 to index 100#now start from 858
start_index=7
end_index=10
for location in final_locations[start_index:end_index]:
    print(location,(np.where(final_locations == location))[0][0])
    print('==========')
    options=check_locations(location)
    list_of_options=[]

    for i in options:
        list_of_options.append(i.text)

    if list_of_options==no_suggestion_list:#means no city
        
        no_data_found.append(location)
    else:
        for real_location in list_of_options:
            if real_location not in final_req.columns:#means it in not already in data
                if not 'Puerto Rico'in real_location:#means no piero data
                    try:
                        get_data(real_location)
                    except: 
                        problem_list.append(real_location)
                        continue


ADA 7
ADAIR 8
ADAMS 9


In [259]:
final_req.to_csv(r'rows_6954.csv')

In [262]:
no_data_found

['Bee Branch',
 'Bee Spring',
 'Beebe Plain',
 'Beech Bluff',
 'Beech Bottom',
 'Beech Creek',
 'Beech Island',
 'Beecher City',
 'Beecher Falls',
 'Beechmont',
 'Beedeville',
 'Beeler',
 'Beemer',
 'Beersheba Springs',
 'Beeson',
 'Beetown',
 'Beggs',
 'Beirne',
 'Bejou',
 'Bel Alton',
 'Belcamp',
 'Belcourt',
 'Belden',
 'Beldenville',
 'Belews Creek',
 'Belfair',
 'Belfield',
 'Belford',
 'Belfry',
 'Belgium',
 'Belgrade Lakes',
 'Belhaven',
 'Belington',
 'Bell Buckle',
 'Bellamy',
 'Bellarthur',
 'Belle Center',
 'Belle Mead',
 'Belle Mina',
 'Belle Rive',
 'Belle Rose',
 'Belle Valley',
 'Belle Vernon',
 'Belleair Beach',
 'Bellemont',
 'Bellerose',
 'Bellerose Village',
 'Bellmont',
 'Bellona',
 'Bellows Falls',
 'Bellport',
 'Bells',
 'Bellvale',
 'Bellville',
 'Bellvue',
 'Belmond',
 'Belmore',
 'Belsano',
 'Belspring',
 'Belva',
 'Belvedere Tiburon',
 'Belvidere Center',
 'Belview',
 'Belvue',
 'Belzoni',
 'Bement',
 'Bemus Point',
 'Ben Bolt',
 'Ben Franklin',
 'Ben Hur',
 '

In [ ]:
#the first data will come in pd1 and now im saving data in final_req
no_dat

In [ ]:
itemindex = numpy.where(final_locations==)

In [ ]:
#extra

In [41]:
final_req.head(5)

,Fact,Xpath,Mississippi,"Mississippi County, Arkansas","Mississippi County, Missouri","Missoula city, Montana","Missoula County, Montana","Mitchell city, South Dakota","Mitchellville CDP, Maryland","Mitchell County, Georgia",...,"Zebulon town, North Carolina","Zeeland city, Michigan","Zeeland charter township, Ottawa County, Michigan","Zephyrhills city, Florida","Zephyrhills South CDP, Florida","Zephyrhills West CDP, Florida","Zimmerman city, Minnesota","Zion city, Illinois","Zionsville town, Indiana","Zuni Pueblo CDP, New Mexico"
0,"Population estimates, July 1, 2019, (V2019)","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody...","2,976,149","40,651","13,180","75,516","119,600","15,679",X,"21,863",...,"5,917","5,536","11,564","16,456",X,X,"5,995","23,487","28,357",X
1,"Population estimates base, April 1, 2010, (V2...","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody...","2,968,130","46,481","14,376","67,358","109,296","15,285",X,"23,501",...,"4,459","5,502","9,971","13,859",X,X,"5,238","24,394","24,392",X
2,"Population, percent change - April 1, 2010 (es...","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody...",0.3%,-12.5%,-8.3%,12.1%,9.4%,2.6%,X,-7.0%,...,32.7%,0.6%,16.0%,18.7%,X,X,14.5%,-3.7%,16.3%,X
3,"Population, Census, April 1, 2010","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody...","2,967,297","46,480","14,358","66,788","109,299","15,254","10,967","23,498",...,"4,433","5,504","9,971","13,288","5,276","5,865","5,228","24,413","14,160","6,302"
4,"Persons under 5 years, percent","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody...",6.2%,7.2%,5.3%,5.1%,5.0%,5.8%,6.1%,5.8%,...,11.1%,9.0%,6.3%,4.8%,4.0%,2.7%,6.9%,6.8%,7.4%,5.4%


In [42]:
final_req.to_csv(r'4761_rows.csv')

In [43]:
df1=final_req

In [44]:
df2=pd.read_csv(r'6954_rows.csv')

In [45]:
df1.drop(['Fact','Xpath'], axis=1, inplace=True)

In [47]:
df3=pd.concat([df1, df2], axis=1)

In [48]:
df3

,Mississippi,"Mississippi County, Arkansas","Mississippi County, Missouri","Missoula city, Montana","Missoula County, Montana","Mitchell city, South Dakota","Mitchellville CDP, Maryland","Mitchell County, Georgia","Mitchell County, Iowa","Mitchell County, Kansas",...,"Miner County, South Dakota","Mineral County, Colorado","Mineral County, Montana","Mineral County, Nevada","Mineral County, West Virginia","Mingo County, West Virginia","Minidoka County, Idaho","Minnehaha CDP, Washington","Minnehaha County, South Dakota","Missaukee County, Michigan"
0,"2,976,149","40,651","13,180","75,516","119,600","15,679",X,"21,863","10,586","5,979",...,"2,216",769,"4,397","4,505","26,868","23,424","21,039",X,"193,134",NaN
1,"2,968,130","46,481","14,376","67,358","109,296","15,285",X,"23,501","10,776","6,373",...,"2,389",712,"4,229","4,770","28,225","26,834","20,055",X,"169,474",NaN
2,0.3%,-12.5%,-8.3%,12.1%,9.4%,2.6%,X,-7.0%,-1.8%,-6.2%,...,-7.2%,8.0%,4.0%,-5.6%,-4.8%,-12.7%,4.9%,X,14.0%,NaN
3,"2,967,297","46,480","14,358","66,788","109,299","15,254","10,967","23,498","10,776","6,373",...,"2,389",712,"4,223","4,772","28,212","26,839","20,069","9,771","169,468",NaN
4,6.2%,7.2%,5.3%,5.1%,5.0%,5.8%,6.1%,5.8%,5.9%,6.3%,...,6.6%,4.0%,4.9%,6.0%,5.1%,6.3%,7.7%,7.1%,7.5%,NaN
5,23.5%,26.1%,20.8%,17.7%,18.6%,22.2%,21.5%,22.4%,23.7%,22.7%,...,24.0%,14.0%,18.2%,19.6%,19.8%,22.4%,28.6%,22.1%,25.2%,NaN
6,16.4%,15.0%,18.2%,12.5%,16.2%,16.9%,14.4%,16.9%,21.4%,24.2%,...,21.0%,33.6%,29.6%,25.4%,22.1%,19.8%,16.2%,12.0%,13.5%,NaN
7,51.5%,51.3%,45.9%,49.7%,50.0%,50.2%,54.1%,48.3%,50.3%,49.6%,...,48.1%,49.7%,48.8%,50.5%,50.2%,50.6%,50.0%,49.0%,49.4%,NaN
8,59.1%,61.8%,73.1%,91.2%,91.7%,92.0%,9.5%,49.3%,97.9%,96.7%,...,96.4%,97.0%,94.0%,68.6%,94.6%,96.3%,94.2%,79.2%,86.2%,NaN
9,37.8%,35.6%,24.8%,0.6%,0.5%,1.0%,81.0%,48.0%,0.6%,0.3%,...,0.8%,0.3%,0.5%,4.8%,3.0%,2.0%,0.8%,2.9%,6.1%,NaN


In [49]:
#returns column containg NAN
nan_values = df3.isna()
nan_columns = nan_values.any()

columns_with_nan = df3.columns[nan_columns].tolist()
print((columns_with_nan))

1360


In [53]:
for i in columns_with_nan:
    print(i)

Anthony city, New Mexico
AARP
ABAC
ABMPS
Alachua city, Florida
Albemarle County, Virginia
AMF
APG
ARP
Arthur County, Nebraska
Ashland town, Middlesex County, Massachusetts
ASU
Atchison County, Kansas
Aaronsburg
Abbot
Abbotsford
Abbott
Abbottstown
Abbyville
Abell
Abercrombie
Aberdeen Proving Ground
Abernant
Abernathy
Abie
Abiquiu
Abita Springs
Abrams
Absaraka
Absarokee
Acampo
Accident
Accord
Accoville
Ace
Achille
Achilles
Ackerly
Ackerman
Ackermanville
Ackley
Ackworth
Acme
Acosta
Acra
Adah
Adairsville
Adairville
Adak
Adamant
Adams Basin
Adams Center
Adams Run
Adamsburg
Adamsville
Addieville
Addington
Addis town, Louisiana
Addy
Addyston
Adell
Adelphia
Adena
Adger
Adin
Adirondack
Adkins
Admire
Adna
Adolph
Adolphus
Adona
Advance
Advent
Afton
Agar
Agate
Agency
Agenda
Ages Brookside
Agness
Agra
Agua Dulce
Aguanga
Aguila
Aguilar
Ah Gwah Ching
Ahmeek
Ahoskie
Ahsahka
Ahwahnee
Ailey
Aimwell
Ainsworth
Airville
Ajo
Akaska
Akeley
Akers
Akiachak
Akiak
Akutan
Aladdin
Alakanuk
Alamo CDP, California
Al

In [52]:
type(columns_with_nan)

list

In [71]:
#now all values which could not be taken will ,be scrapped in finalreq
final_req.head(5)

,Fact,Xpath,"Anthony city, New Mexico","Alachua city, Florida","Albemarle County, Virginia","Arthur County, Nebraska","Ashland town, Middlesex County, Massachusetts","Atchison County, Kansas","Alamo CDP, California","Ballard County, Kentucky",...,"McLean County, Kentucky","McLeod County, Minnesota","McMullen County, Texas","McPherson County, Nebraska","Meagher County, Montana","Menard County, Texas","Menifee County, Kentucky","Metcalfe County, Kentucky","Mills County, Texas","Missaukee County, Michigan"
0,"Population estimates, July 1, 2019, (V2019)","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody...","9,239","9,899","109,330",463,"17,807","16,073",X,"7,888",...,"9,207","35,893",743,494,"1,862","2,138","6,489","10,071","4,873","15,118"
1,"Population estimates base, April 1, 2010, (V2...","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody...","9,467","9,041","98,998",460,"16,615","16,921",X,"8,246",...,"9,540","36,645",707,539,"1,891","2,242","6,309","10,116","4,941","14,851"
2,"Population, percent change - April 1, 2010 (es...","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody...",-2.4%,9.5%,10.4%,0.7%,7.2%,-5.0%,X,-4.3%,...,-3.5%,-2.1%,5.1%,-8.3%,-1.5%,-4.6%,2.9%,-0.4%,-1.4%,1.8%
3,"Population, Census, April 1, 2010","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody...",X,"9,059","98,970",460,"16,593","16,924","14,570","8,249",...,"9,531","36,651",707,539,"1,891","2,242","6,306","10,099","4,936","14,849"
4,"Persons under 5 years, percent","//*[@id=""qfcontent""]/div[5]/div/table[2]/tbody...",10.1%,9.2%,5.2%,5.2%,6.3%,5.6%,2.2%,5.0%,...,5.6%,5.7%,5.4%,2.8%,4.7%,4.0%,4.8%,6.1%,4.2%,6.3%


In [72]:
#returns column containg NAN
nan_values = final_req.isna()
nan_columns = nan_values.any()

columns_with_nan = final_req.columns[nan_columns].tolist()
print((columns_with_nan))

[]


In [73]:
final_req.drop(['Fact','Xpath'], axis=1, inplace=True)

,"Anthony city, New Mexico","Alachua city, Florida","Albemarle County, Virginia","Arthur County, Nebraska","Ashland town, Middlesex County, Massachusetts","Atchison County, Kansas","Alamo CDP, California","Ballard County, Kentucky","Banner County, Nebraska","Latrobe borough, Pennsylvania",...,"McLean County, Kentucky","McLeod County, Minnesota","McMullen County, Texas","McPherson County, Nebraska","Meagher County, Montana","Menard County, Texas","Menifee County, Kentucky","Metcalfe County, Kentucky","Mills County, Texas","Missaukee County, Michigan"
0,"9,239","9,899","109,330",463,"17,807","16,073",X,"7,888",745,"7,830",...,"9,207","35,893",743,494,"1,862","2,138","6,489","10,071","4,873","15,118"
1,"9,467","9,041","98,998",460,"16,615","16,921",X,"8,246",690,"8,333",...,"9,540","36,645",707,539,"1,891","2,242","6,309","10,116","4,941","14,851"
2,-2.4%,9.5%,10.4%,0.7%,7.2%,-5.0%,X,-4.3%,8.0%,-6.0%,...,-3.5%,-2.1%,5.1%,-8.3%,-1.5%,-4.6%,2.9%,-0.4%,-1.4%,1.8%
3,X,"9,059","98,970",460,"16,593","16,924","14,570","8,249",690,NA,...,"9,531","36,651",707,539,"1,891","2,242","6,306","10,099","4,936","14,849"
4,10.1%,9.2%,5.2%,5.2%,6.3%,5.6%,2.2%,5.0%,7.5%,4.8%,...,5.6%,5.7%,5.4%,2.8%,4.7%,4.0%,4.8%,6.1%,4.2%,6.3%
5,34.1%,28.0%,19.7%,25.3%,23.0%,22.8%,23.9%,20.9%,24.4%,19.9%,...,23.5%,22.8%,19.8%,18.2%,18.7%,18.1%,18.4%,23.4%,20.4%,22.9%
6,8.0%,13.3%,19.3%,21.4%,14.8%,17.6%,22.4%,21.9%,24.0%,19.6%,...,20.0%,19.1%,25.4%,24.1%,31.6%,32.7%,21.4%,19.4%,29.1%,20.7%
7,52.1%,53.7%,52.1%,51.0%,51.1%,51.4%,48.3%,50.3%,51.5%,48.8%,...,50.8%,50.0%,47.2%,50.8%,48.8%,50.1%,49.4%,50.6%,50.0%,49.6%
8,89.2%,68.0%,81.8%,97.2%,82.9%,90.9%,85.2%,93.2%,97.6%,97.4%,...,97.1%,96.6%,93.5%,98.4%,96.8%,95.1%,95.4%,96.6%,96.1%,96.2%
9,0.3%,28.3%,9.7%,0.0%,2.7%,4.8%,0.8%,3.4%,1.3%,0.5%,...,0.9%,0.8%,2.4%,0.4%,0.5%,1.7%,2.8%,1.8%,1.2%,0.6%


In [75]:
df3.head()

,Mississippi,"Mississippi County, Arkansas","Mississippi County, Missouri","Missoula city, Montana","Missoula County, Montana","Mitchell city, South Dakota","Mitchellville CDP, Maryland","Mitchell County, Georgia","Mitchell County, Iowa","Mitchell County, Kansas",...,"Miner County, South Dakota","Mineral County, Colorado","Mineral County, Montana","Mineral County, Nevada","Mineral County, West Virginia","Mingo County, West Virginia","Minidoka County, Idaho","Minnehaha CDP, Washington","Minnehaha County, South Dakota","Missaukee County, Michigan"
0,"2,976,149","40,651","13,180","75,516","119,600","15,679",X,"21,863","10,586","5,979",...,"2,216",769,"4,397","4,505","26,868","23,424","21,039",X,"193,134",NaN
1,"2,968,130","46,481","14,376","67,358","109,296","15,285",X,"23,501","10,776","6,373",...,"2,389",712,"4,229","4,770","28,225","26,834","20,055",X,"169,474",NaN
2,0.3%,-12.5%,-8.3%,12.1%,9.4%,2.6%,X,-7.0%,-1.8%,-6.2%,...,-7.2%,8.0%,4.0%,-5.6%,-4.8%,-12.7%,4.9%,X,14.0%,NaN
3,"2,967,297","46,480","14,358","66,788","109,299","15,254","10,967","23,498","10,776","6,373",...,"2,389",712,"4,223","4,772","28,212","26,839","20,069","9,771","169,468",NaN
4,6.2%,7.2%,5.3%,5.1%,5.0%,5.8%,6.1%,5.8%,5.9%,6.3%,...,6.6%,4.0%,4.9%,6.0%,5.1%,6.3%,7.7%,7.1%,7.5%,NaN


In [76]:
df4=pd.concat([df3, final_req], axis=1)

In [77]:
df4.head()

,Mississippi,"Mississippi County, Arkansas","Mississippi County, Missouri","Missoula city, Montana","Missoula County, Montana","Mitchell city, South Dakota","Mitchellville CDP, Maryland","Mitchell County, Georgia","Mitchell County, Iowa","Mitchell County, Kansas",...,"McLean County, Kentucky","McLeod County, Minnesota","McMullen County, Texas","McPherson County, Nebraska","Meagher County, Montana","Menard County, Texas","Menifee County, Kentucky","Metcalfe County, Kentucky","Mills County, Texas","Missaukee County, Michigan"
0,"2,976,149","40,651","13,180","75,516","119,600","15,679",X,"21,863","10,586","5,979",...,"9,207","35,893",743,494,"1,862","2,138","6,489","10,071","4,873","15,118"
1,"2,968,130","46,481","14,376","67,358","109,296","15,285",X,"23,501","10,776","6,373",...,"9,540","36,645",707,539,"1,891","2,242","6,309","10,116","4,941","14,851"
2,0.3%,-12.5%,-8.3%,12.1%,9.4%,2.6%,X,-7.0%,-1.8%,-6.2%,...,-3.5%,-2.1%,5.1%,-8.3%,-1.5%,-4.6%,2.9%,-0.4%,-1.4%,1.8%
3,"2,967,297","46,480","14,358","66,788","109,299","15,254","10,967","23,498","10,776","6,373",...,"9,531","36,651",707,539,"1,891","2,242","6,306","10,099","4,936","14,849"
4,6.2%,7.2%,5.3%,5.1%,5.0%,5.8%,6.1%,5.8%,5.9%,6.3%,...,5.6%,5.7%,5.4%,2.8%,4.7%,4.0%,4.8%,6.1%,4.2%,6.3%


In [101]:
#returns column containg NAN
nan_values = df6.isna()
nan_columns = nan_values.any()

columns_with_nan = df6.columns[nan_columns].tolist()
print((columns_with_nan))

['Anthony city, New Mexico', 'Arthur County, Nebraska', 'Ballard County, Kentucky', 'Banner County, Nebraska', 'Latrobe borough, Pennsylvania', 'Lawrence County, Kentucky', 'Lee County, Kentucky', 'Bedford town, Virginia', 'Bella Vista city, Arkansas', 'Blaine County, Nebraska', 'Borden County, Texas', 'Bracken County, Kentucky', 'Briscoe County, Texas', 'Brookhaven city, Georgia', 'Calhoun County, West Virginia', 'Carlisle County, Kentucky', 'Carroll County, Mississippi', 'Catron County, New Mexico', 'Cibola County, New Mexico', 'Crockett County, Texas', 'Cave Springs city, Arkansas', 'De Baca County, New Mexico', 'Denali Borough, Alaska', 'Dewey County, Oklahoma', 'Dickens County, Texas', 'Duval County, Texas', 'Diamondhead city, Mississippi', 'Edmonson County, Kentucky', 'Edwards County, Illinois', 'Edwards County, Texas', 'Elk County, Kansas', 'Elliott County, Kentucky', 'Esmeralda County, Nevada', 'Eureka County, Nevada', 'Estero village, Florida', 'Fisher County, Texas', 'Fox Cro

In [84]:
#delete all columns with nan
df5=df4.dropna(axis=1, how='all')

In [111]:
df5.head()#contains all data

,Mississippi,"Mississippi County, Arkansas","Mississippi County, Missouri","Missoula city, Montana","Missoula County, Montana","Mitchell city, South Dakota","Mitchellville CDP, Maryland","Mitchell County, Georgia","Mitchell County, Iowa","Mitchell County, Kansas",...,"McLean County, Kentucky","McLeod County, Minnesota","McMullen County, Texas","McPherson County, Nebraska","Meagher County, Montana","Menard County, Texas","Menifee County, Kentucky","Metcalfe County, Kentucky","Mills County, Texas","Missaukee County, Michigan"
0,"2,976,149","40,651","13,180","75,516","119,600","15,679",X,"21,863","10,586","5,979",...,"9,207","35,893",743,494,"1,862","2,138","6,489","10,071","4,873","15,118"
1,"2,968,130","46,481","14,376","67,358","109,296","15,285",X,"23,501","10,776","6,373",...,"9,540","36,645",707,539,"1,891","2,242","6,309","10,116","4,941","14,851"
2,0.3%,-12.5%,-8.3%,12.1%,9.4%,2.6%,X,-7.0%,-1.8%,-6.2%,...,-3.5%,-2.1%,5.1%,-8.3%,-1.5%,-4.6%,2.9%,-0.4%,-1.4%,1.8%
3,"2,967,297","46,480","14,358","66,788","109,299","15,254","10,967","23,498","10,776","6,373",...,"9,531","36,651",707,539,"1,891","2,242","6,306","10,099","4,936","14,849"
4,6.2%,7.2%,5.3%,5.1%,5.0%,5.8%,6.1%,5.8%,5.9%,6.3%,...,5.6%,5.7%,5.4%,2.8%,4.7%,4.0%,4.8%,6.1%,4.2%,6.3%


In [96]:
xdf5_copy=df5

In [100]:
df6=df5.loc[:, ~df5.isnull().all()]

In [104]:
df5['Anthony city, New Mexico'].head(5)

,"Anthony city, New Mexico","Anthony city, New Mexico"
0,"9,239","9,239"
1,"9,467","9,467"
2,-2.4%,-2.4%
3,X,X
4,10.1%,10.1%


# df5 conatins all data

In [110]:
del df5['Xpath']

In [113]:
df6 = df5.loc[:,~df5.columns.duplicated()]

In [114]:
df6.head(5)#conatins all data

,Mississippi,"Mississippi County, Arkansas","Mississippi County, Missouri","Missoula city, Montana","Missoula County, Montana","Mitchell city, South Dakota","Mitchellville CDP, Maryland","Mitchell County, Georgia","Mitchell County, Iowa","Mitchell County, Kansas",...,"Lapeer city, Michigan","Lakewood township, Ocean County, New Jersey","Levelland city, Texas","Lincroft CDP, New Jersey","Lionville CDP, Pennsylvania","Lockhart CDP, Florida","London city, Kentucky","Lone Grove city, Oklahoma","Lumberton city, Texas","Missaukee County, Michigan"
0,"2,976,149","40,651","13,180","75,516","119,600","15,679",X,"21,863","10,586","5,979",...,"8,535","106,300","13,502",X,X,X,"8,068","5,121","13,073","15,118"
1,"2,968,130","46,481","14,376","67,358","109,296","15,285",X,"23,501","10,776","6,373",...,"8,826","92,799","13,555",X,X,X,"7,985","4,829","11,972","14,851"
2,0.3%,-12.5%,-8.3%,12.1%,9.4%,2.6%,X,-7.0%,-1.8%,-6.2%,...,-3.3%,14.5%,-0.4%,X,X,X,1.0%,6.0%,9.2%,1.8%
3,"2,967,297","46,480","14,358","66,788","109,299","15,254","10,967","23,498","10,776","6,373",...,"8,841","92,843","13,542","6,135","6,189","13,060","7,993","5,054","11,943","14,849"
4,6.2%,7.2%,5.3%,5.1%,5.0%,5.8%,6.1%,5.8%,5.9%,6.3%,...,3.8%,17.5%,7.0%,5.8%,6.5%,7.2%,6.2%,7.9%,6.6%,6.3%


In [115]:
my_final_df = df6.reindex(sorted(df6.columns), axis=1)

In [120]:
my_final_df.head()

,Fact,,"Abbeville County, South Carolina","Abbeville city, Louisiana","Abbeville city, South Carolina","Aberdeen city, Maryland","Aberdeen city, Mississippi","Aberdeen city, South Dakota","Aberdeen city, Washington","Aberdeen town, North Carolina",...,"Zeeland charter township, Ottawa County, Michigan","Zeeland city, Michigan","Zephyrhills South CDP, Florida","Zephyrhills West CDP, Florida","Zephyrhills city, Florida","Ziebach County, South Dakota","Zimmerman city, Minnesota","Zion city, Illinois","Zionsville town, Indiana","Zuni Pueblo CDP, New Mexico"
0,"Population estimates, July 1, 2019, (V2019)","328,239,523","24,527","12,038","5,014","16,019","5,205","28,257","16,756","7,988",...,"11,564","5,536",X,X,"16,456","2,756","5,995","23,487","28,357",X
1,"Population estimates base, April 1, 2010, (V2...","308,758,105","25,416","12,227","5,254","14,982","5,612","26,112","16,872","6,369",...,"9,971","5,502",X,X,"13,859","2,801","5,238","24,394","24,392",X
2,"Population, percent change - April 1, 2010 (es...",6.3%,-3.5%,-1.5%,-4.6%,6.9%,-7.3%,8.2%,-0.7%,25.4%,...,16.0%,0.6%,X,X,18.7%,-1.6%,14.5%,-3.7%,16.3%,X
3,"Population, Census, April 1, 2010","308,745,538","25,417","12,257","5,237","14,959","5,612","26,091","16,896","6,350",...,"9,971","5,504","5,276","5,865","13,288","2,801","5,228","24,413","14,160","6,302"
4,"Persons under 5 years, percent",6.0%,4.7%,8.5%,7.7%,6.0%,6.8%,6.2%,8.4%,9.2%,...,6.3%,9.0%,4.0%,2.7%,4.8%,5.5%,6.9%,6.8%,7.4%,5.4%


In [119]:
#adding new clumn at first index
my_final_df.insert(loc=0, column='Fact', value=final_req['Fact'])

In [118]:
final_req.Fact.head()

0         Population estimates, July 1, 2019,  (V2019)
1    Population estimates base, April 1, 2010,  (V2...
2    Population, percent change - April 1, 2010 (es...
3                    Population, Census, April 1, 2010
4                       Persons under 5 years, percent
Name: Fact, dtype: object

In [121]:
my_final_df.to_csv(r'Final_US_data_from_census_gov.csv')#finally data into csv